In [1]:
import numpy as np
import RNTK_NEW
import jax
import symjax
import symjax.tensor as T
import math

In [2]:
L = 2
Ti = 4
TiPrime = 4
qmatrix = np.zeros([L,Ti],dtype = object)
DATA = T.Placeholder((Ti, 8), 'float32')
DATAPRIME = T.Placeholder((TiPrime, 8), 'float32')
X = DATA[:,0]
tx = X*X[:, None]
n = tx.shape[0]
su = 1
sb = 1
sh = 1
sq = 1
sw = 1

In [3]:
def normv(x):
    return T.sqrt(T.sum(T.power(x, 2)))
def capQ(lq):
    return T.full((2,2), lq)

In [4]:
def VTq(M):
    A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
    # of all pairs of data in the data set
    B = A * A[:, None]
    C = T.sqrt(B)  # in R^{n*n}
    D = M / C  # this is lamblda in ReLU analyrucal formula
    E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
    F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
    G = (np.pi - T.arccos(E)) / (2 * np.pi)
    return F,G

In [19]:
for t in range(0, Ti):
    for l in range(0, L):
        print(l, t)
        if ((t == l) & (l == 0)):
            #assuming ||x|| means the norm of that vector
            qmatrix[l, t] = su**2 * normv(X[0])**2 + sb**2 + sh ** 2

        if ((t == 0) & (l > t)):

            S,D = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = su**2 * S + sb**2

        if ((t > l) & (l == 0)):
            S,D = VTq(capQ(qmatrix[l,t-1]))
            qmatrix[l, t] = sw**2 * S + su**2 * normv(X[t])**2 + sb**2

        if ((t > l) & (l > 0)):
            S_left, D_left = VTq(capQ(qmatrix[l,t-1]))
            S_right, D_right = VTq(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = sw**2 * S_left + su**2 * D_right + sb**2

        print(qmatrix[l, t])


0 0
Op(name=add_30, fn=add, shape=(), dtype=float32, scope=/default)
1 0
Op(name=add_32, fn=add, shape=(2, 2), dtype=float32, scope=/default)
0 1
Op(name=add_35, fn=add, shape=(2, 2), dtype=float32, scope=/default)
1 1
0
0 2
Op(name=add_38, fn=add, shape=(2, 2), dtype=float32, scope=/default)
1 2
Op(name=add_42, fn=add, shape=(2, 2), dtype=float32, scope=/default)
0 3
Op(name=add_45, fn=add, shape=(2, 2), dtype=float32, scope=/default)
1 3
Op(name=add_49, fn=add, shape=(2, 2), dtype=float32, scope=/default)


In [6]:
#MAKE ALGORITHM 2

In [17]:
def Kfunc(l, t, tp):
    elem1 = qmatrix[l,t]
    elem2 = qmatrix[l, tp]
    retmatrix = T.full((1,4), elem1)
    T.index_update(retmatrix, 3, elem2)
    return T.reshape(retmatrix, (2,2))


In [18]:
switch_flag = 1
dim_1_i = dim_1 = 3
dim_2 = 3
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

thetavalue = 0
kmatrix = 0

lambdamatrix = np.zeros([L,Ti, TiPrime],dtype = object)
phimatrix = np.zeros([L,Ti, TiPrime],dtype = object)

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    # print("dim_1_i", dim_1_i, "dim_2_i", dim_2_i, switch_flag)
    while ((tiprime <= dim_1) & (ti <= dim_2)):
        print("inner iteration - ", tiprime, ti) #// DIM 1 IS T PRIME, DIM 2 IS T
        xi = DATA[:, ti] # 1 is t
        xpi = DATAPRIME[:, tiprime]
        if ((ti > 0) & (tiprime > 0)):
            S, D = VTq(Kfunc(0, ti-1, tiprime-1))
            lambdamatrix[0, ti, tiprime] = sw ** 2 * S + su * (xi*xpi[:, None]) + sb ** 2
            phimatrix[0, ti, tiprime] = lambdamatrix[0, ti, tiprime] + sw ** 2 * phimatrix[0, ti - 1, tiprime - 1] * D
        else:
            phimatrix[0,ti,tiprime] = lambdamatrix[0,ti,tiprime] = su ** 2 * (xi*xpi[:, None]) + sb

        for l in range(1, L):
            if ((ti > 0) & (tiprime > 0)):
                Sold, Dold = VTq(Kfunc(l-1, ti, tiprime))
                Sprev, Dprev = VTq(Kfunc(l, ti-1, tiprime-1))
                lambdamatrix[l,ti,tiprime] = sw**2 * Sprev + sb**2 + su**2 * Sold  + sb**2
                phimatrix[l,ti,tiprime] = lambdamatrix[l,ti,tiprime] + sw**2*phimatrix[l,ti-1,tiprime-1]*Dprev + su**2*phimatrix[l-1,ti,tiprime]*Dold
            else:
                S, D = VTq(Kfunc(l-1, ti, tiprime))
                lambdamatrix[l,ti,tiprime] = su**2 * S + sb**2
                phimatrix[l,ti,tiprime] = lambdamatrix[l,ti,tiprime] + su**2 * phimatrix[l-1,ti,tiprime] * D

        S, D = VTq(Kfunc(L, ti, tiprime))
        kmatrix += sv**2 * S
        thetamatrix += sv**2 * S + sv**2 * phimatrix[L,ti,tiprime] * D

        tiprime+=1
        ti+=1

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

    

3 0 1
inner iteration -  3 0
inner iteration -  2 0
inner iteration -  3 1
Op(name=add_1, fn=add, shape=(), dtype=float32, scope=/default)
Op(name=add_9, fn=add, shape=(2, 2), dtype=float32, scope=/default)


ValueError: Incompatible shapes for broadcasting: (2, 2) and requested shape (4,)